# Heart Rate Forecasting - Time-MoE

In [ ]:
# Setup: clonar repo e instalar dependencias
!git clone https://github.com/AllanDBB/heartrate-forecasting.git
%cd heartrate-forecasting
!pip install -q -r requirements.txt
# Forzar numpy compatible y reiniciar runtime para evitar incompatibilidad binaria
!pip install -q "numpy<2.0"
import os; os.kill(os.getpid(), 9)  # Reinicia el kernel automáticamente

In [ ]:
# Verificar GPU
import torch
print('GPU available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('Device:', torch.cuda.get_device_name(0))
    print(f'VRAM libre: {torch.cuda.mem_get_info()[0]/1e9:.2f} GB')

In [ ]:
# Cargar y preparar datos
import utils
from sklearn.model_selection import train_test_split

folder_path = 'dataset/'
dfSelectedMean = utils.loadAllFiles(folder_path)

inp = 200
out = 200

df_70, df_30 = utils.selectRandomColumns(dfSelectedMean)

pathEst_70 = 'dataset/values_deses_70.csv'
pathEst_30 = 'dataset/values_deses_30.csv'

df_scaled_70, values_deses_70 = utils.estandarizar(df_70, pathEst_70)
df_scaled_30, values_deses_30 = utils.estandarizar(df_30, pathEst_30)

X_train, y_train, ids_list_tr = utils.series_to_supervised_matrix(
    df_scaled_70.iloc[:, :-2], input_size=inp, output_size=out
)
X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

X_test, y_test, ids_list_te = utils.series_to_supervised_matrix(
    df_scaled_30.iloc[:, :-2], input_size=inp, output_size=out
)

print('Datasets splitted and ready!')
print('Shapes:', X_tr.shape, y_tr.shape, X_va.shape, y_va.shape, X_test.shape, y_test.shape)

In [ ]:
# Cargar wrapper de Time-MoE
import os, sys
os.chdir('/content/heartrate-forecasting')
sys.path.insert(0, '/content/heartrate-forecasting')

from wrappers.TimeMoESupervisedWrapper import TimeMoESupervisedWrapper

wrapper = TimeMoESupervisedWrapper(config_path='configs/timemoe_config.yaml')

In [ ]:
# Smoke test con 5 muestras
y_pred_smoke = wrapper.predict(X_test[:5])
print('Shape predicción:', y_pred_smoke.shape)  # esperado: (5, 200)

In [ ]:
# Evaluación sobre test set
metrics_test = wrapper.evaluate(X_test, y_test)

In [ ]:
# Evaluación sobre validation set
metrics_val = wrapper.evaluate(X_va, y_va)